In [1]:
import re 

split_words = [
    " de ",
    " d'"
]

unsplitable_words = [
    "huile d'olive",
    "herbes de provence"
]

prefix = "* "

units = [
    "g",
    'cl',
]

def p_u_word(ingredient: str):
    ingredient = ingredient.lower()
    for unsplitable_word in unsplitable_words:
        if unsplitable_word in ingredient.lower():
            si = " ".join(ingredient.split(unsplitable_word)).strip()
            if not si:
                return f"@{unsplitable_word}{{}}"
    return ""

def p_single_word(ingredient: str) -> str:
    if not " " in ingredient:
        return f"@{ingredient}"
    return ""

def p_s_word(ingredient: str):
    for word in split_words:
        if word in ingredient:
            split_ingredients = ingredient.split(word)
            ingredient_name = word.join(split_ingredients[1:]).strip()
            ingredient_q_u = split_ingredients[0].strip()
            formated_q_u = p_q_u(ingredient_q_u)
            return f"@{ingredient_name}{formated_q_u}"
    return ""

def p_q_u(q_u: str):
    """process quantity and unit
    """
    q_u_split = q_u.split(" ")
    if len(q_u_split) == 1:
        return f"{{{q_u}}}"
    q = q_u_split[0]
    u = " ".join(q_u_split[1:])
    return f"{{{q}%{u}}}"

def p_start_int(ingredient: str) -> str:
    i_split = ingredient.split(" ")
    if re.match(r"\d+", i_split[0]):
        if i_split[1] in units:
            ingred = " ".join(i_split[2:])
            return f"@{ingred}{{{i_split[0]}%{i_split[1]}}}"
        ingred = " ".join(i_split[1:])
        return f"@{ingred}{{{i_split[0]}}}"
    return ""

def process_ingredients(ingredients: str) -> str:
    out = ""
    for ingredient in ingredients.split("\n"):
        ingredient = ingredient.removeprefix(prefix).strip().lower()
        if not ingredient:
            continue
        if t := p_single_word(ingredient):
            out += t + "\n"
            continue
        if t := p_u_word(ingredient):
            out += t + "\n"
            continue
        if t := p_s_word(ingredient):
            out += t + "\n"
            continue
        if t := p_start_int(ingredient):
            out += t + "\n"
            continue
        out += f"@{ingredient}{{}}\n"
    return out

In [22]:
ingredients="""
* 1 yaourt et son pot vide
* 1.5 pot  de sucre en poudre
* 3 pots de farine
* 1/2 pot d'huile d'arachide
* 3 œufs entiers
* 1/2 sachet de levure
* 1 pincée de sel
* Un peu de beurre pour le moule ou du papier sulfurisé
"""

protocol = """
* Préchauffer le four à 180° (th 6).
* Vider le yaourt dans un plat creux. Laver le pot et l'essuyer. Verser le sucre, l'huile et le sel dans le plat. Ajouter la farine et la levure puis les œufs entiers. Bien mélanger.
* Beurrer le moule (rond ou à cake) ou le papier de cuisson disposé dans le moule. Y verser la pâte et faire cuire au four pendant 45mn.
"""

In [23]:
out = process_ingredients(ingredients)
map_ingredient_to_cooklang = {
    l[1:].split("{")[0]: l for l in out.splitlines()
}
map_ingredient_to_cooklang

{'yaourt et son pot vide': '@yaourt et son pot vide{1}',
 'sucre en poudre': '@sucre en poudre{1.5%pot}',
 'farine': '@farine{3%pots}',
 "huile d'arachide": "@huile d'arachide{1/2%pot}",
 'œufs entiers': '@œufs entiers{3}',
 'levure': '@levure{1/2%sachet}',
 'sel': '@sel{1%pincée}',
 'beurre pour le moule ou du papier sulfurisé': '@beurre pour le moule ou du papier sulfurisé{un%peu}'}

In [24]:
processed_ingredients = []
for step in protocol.split("\n"):
    step = step.strip().removeprefix("* ")
    if not step:
        continue
    for ingredient in map_ingredient_to_cooklang.keys():
        if ingredient.lower() in step and ingredient not in processed_ingredients:
            step = step.replace(ingredient, map_ingredient_to_cooklang[ingredient])
            processed_ingredients.append(ingredient)

    print(step)

print("\n".join(v for k,v in map_ingredient_to_cooklang.items() if k not in processed_ingredients))


Préchauffer le four à 180° (th 6).
Vider le yaourt dans un plat creux. Laver le pot et l'essuyer. Verser le sucre, l'huile et le @sel{1%pincée} dans le plat. Ajouter la @farine{3%pots} et la @levure{1/2%sachet} puis les @œufs entiers{3}. Bien mélanger.
Beurrer le moule (rond ou à cake) ou le papier de cuisson disposé dans le moule. Y verser la pâte et faire cuire au four pendant 45mn.
@yaourt et son pot vide{1}
@sucre en poudre{1.5%pot}
@huile d'arachide{1/2%pot}
@beurre pour le moule ou du papier sulfurisé{un%peu}
